# Retrieving Stream Metadata 

<br>

### Imports

In [1]:
import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import functools
from ipypb import track

from NGDataPortal import Wrapper, stream_to_id_map

<br>

### Helper Functions

In [2]:
def try_accept(exception=np.nan):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except:
                return exception
        return wrapper
    return decorator

<br>

### Collecting Stream Data

We start by extracting the labels for the streams

In [8]:
streams = list(stream_to_id_map.keys())

streams[:5]

['contracted-energy-volumes-and-data',
 'firm-frequency-response-auction-results',
 'fast-reserve-tender-reports',
 'balancing-services-charging-report-bcr',
 'current-balancing-services-use-of-system-bsuos-data']

<br>

Next we create a function which will allow us to extract an example row from each returned dataframe

In [4]:
df_to_example_row = lambda df: df.iloc[0].to_dict() 

stream = streams[0]
wrapper = Wrapper(stream)

df = wrapper.query_API()
example_row = df_to_example_row(df)

example_row

{'_id': 1,
 'Date': '2019-04-11T00:00:00',
 'SP': 48,
 'Volume Fixed Price (MW)': 0,
 'Volume Indexed (MW)': 0}

<br>

We're now ready to cycle through each of the streams and repeat the row extraction step

In [5]:
def stream_2_updated_dict(stream, example_rows):
    try:
        wrapper = Wrapper(stream)
        df = wrapper.query_API()
        example_row = df_2_example_row(df)
        
    except:
        example_row = np.nan
    
    example_rows[stream] = example_row
    
    return example_rows

example_rows = dict()

for stream in track(streams):
    example_rows = stream_2_updated_dict(stream, example_rows)

 [████████████████████████████████████████████████████████████] 36/36 [00:09<00:00, 0.24s/it]

<br>

We'll now go through and log any streams which didnt return a dataframe and remove them from the main stream dictionary

In [6]:
null_streams = []

for stream in streams:
    if isinstance(example_rows[stream], float):
        null_streams += [stream]
        example_rows.pop(stream)
        
null_streams

['fast-reserve-tender-reports',
 'balancing-services-charging-report-bcr',
 'mbss',
 'firm-frequency-response-market-information',
 'balancing-services-use-of-system-bsuos-daily-cost',
 'fast-reserve-market-information-reports',
 'short-term-operating-reserve-stor',
 'historic-demand-data',
 'obligatory-reactive-power-service-orps-utilisation',
 'firm-frequency-response-post-tender-reports']

<br>

Finally we'll output the streams along with the names of the columns in their dataframes

In [7]:
cols = [list(example_row.keys()) for example_row in example_rows.values()]
stream_2_cols = dict(zip(example_rows.keys(), cols))

for key, value in stream_2_cols.items():
    print(f"{key}:\n{', '.join(value)}\n")

contracted-energy-volumes-and-data:
_id, Date, SP, Volume Fixed Price (MW), Volume Indexed (MW)

firm-frequency-response-auction-results:
_id, DATE, EFA, SERVICE ID, BUY QTY LIMIT MW, BUY PRICE LIMIT GBP/MW/H, CLEARED QUANTITY MW, CLEARED PRICE GBP/MW/H

current-balancing-services-use-of-system-bsuos-data:
_id, Settlement Day, Settlement Period, BSUoS Price (£/MWh Hour), Half-hourly Charge, Total Daily BSUoS Charge, Run Type

weekly-wind-availability:
_id, BMU_ID, Week Number, MW

outturn-voltage-costs:
_id, Row Labels, Dumfries & Galloway, E Corridor, E England, East Midlands, Humber, London, Mersey, N Wales, NE Scotland, NW England, NW Scotland, S Central, S Wales, SE England, SW England, W Corridor, W Midlands, W Scotland, South Yorks, Grand Total

bsuos-monthly-cost:
_id, Month, Financial Week, Day, Total Balancing Cost, F = forecast, BM + Trading + SOSO, Non Delivery, Total Balancing Mechanism Costs, Energy Balancing, Total System Balancing, Response, Negative Reserve, Fast Reserv